<a href="https://colab.research.google.com/github/nima-abdpoor/snapfood_satisfaction/blob/main/SnapFood_Satisfaction_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# imports

In [ ]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras import regularizers
from keras.layers import Embedding
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import io
from google.colab import files
import gensim
from gensim.models.word2vec import Word2Vec
#!pip install hazm
#!pip install tensorflow
from hazm import word_tokenize
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, SpatialDropout1D
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import LabelEncoder
import keras
from keras.initializers import Constant
from keras.layers import ReLU
from keras.layers import Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical




     |████████████████████████████████| 316 kB 4.0 MB/s 
     |████████████████████████████████| 233 kB 8.2 MB/s 
     |████████████████████████████████| 1.4 MB 9.3 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394488 sha256=f65962071ec15e3792559df7373f3fe769ddf2d78f23faa5ca570c2f8e813663
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154746 sha256=40295f0c57de558ab94726fe384e58431d1b07d7a3e979cf255f7bdd3f8a62b1
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
  tfds.disable_progress_bar()


In [ ]:
def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

#Setup input pipeline

In [ ]:
dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True )
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteFTXQXN/imdb_reviews-train.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteFTXQXN/imdb_reviews-test.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteFTXQXN/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [ ]:
uploaded = files.upload()


Saving dev.csv to dev.csv
Saving train.csv to train.csv


In [ ]:
train = pd.read_csv(io.BytesIO(uploaded['train.csv']),sep='\t')
dev = pd.read_csv(io.BytesIO(uploaded['dev.csv']), sep='\t')
train.comment = train.comment.astype('str')
train.label = train.label.astype('str')


In [ ]:
train

,Unnamed: 0,comment,label,label_id
0,0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1,1
1,1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0,0
2,2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1,1
3,3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0,0
4,4,شیرینی وانیلی فقط یک مدل بود.,0,0
...,...,...,...,...
56695,56695,یک تیکه کم فرستاده بودن و با تماس من در کمترین...,0,0
56696,56696,عالی بود همه چیز ممنونم پیک هم خیلی مرتب و به ...,0,0
56697,56697,مثل همیشه عالی، من چندمین باره سفارش میدم و هر...,0,0
56698,56698,دلستر استوایی خواسته بودم اما لیمویی فرستادند,0,0


In [ ]:
dev

,Unnamed: 0,comment,label,label_id
0,0,همه ساندویچ‌ها سرد بود متاسفانه در صورتی که فا...,SAD,1
1,1,تشکر، عالی‌تر از همیشه,HAPPY,0
2,2,مرغ و کدو رو دوس نداشتم ولی کلا ساندویچهای خوش...,HAPPY,0
3,3,نمیدونم هزینه بسته بندی به چه دلیل گرفته شد؟؟؟,SAD,1
4,4,یک نوشابه کم ارسال شد و تاخیر فراوان و غذای نس...,SAD,1
...,...,...,...,...
6295,6295,سلام من به فاکتور غذاهایی که سفارش میدم احتیاج...,SAD,1
6296,6296,سایز پیتزا نسبت به سفارشاتی که قبلا گذشتم کم ش...,SAD,1
6297,6297,من قارچ اضافه رو اضافه کرده بودم بودم اما اگر ...,HAPPY,0
6298,6298,همرو بعد ۲ساعت تاخیر اشتباه آوردن پولشم رفت رو...,SAD,1


In [ ]:
labelencoder = LabelEncoder()
train['label'] = labelencoder.fit_transform(train['label'])

In [ ]:
train.comment = train.comment.astype('str')
train.label = train.label.astype('str')

In [ ]:
data = []
#Splitting pd.Series to list
data_to_list = train['comment'].values.tolist()
for i in range(len(data_to_list)):
    data.append(data_to_list[i])

print(data[:5])



['واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح', 'قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از موقع رسید، شما ببین چقدرررررررررررر پلاک خفنههههه، من سالهاست مشتریشونم و سالهاست مزه بهشت میده غذاشون', 'قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط ظاهر فریبنده داره، پرش میکنن کالباس و قارچ', 'عالللی بود همه چه درست و به اندازه و کیفیت خوب، امیداورم همیشه کیفیتتون خوب باشه ما مشتری همیشگی بشیم', 'شیرینی وانیلی فقط یک مدل بود.']


In [ ]:
# converting each sentence to list of words and inserting in sents
sents = [word_tokenize(comment) for comment in data]

# example
sents[45]

['کیک',
 'و',
 'جعبه',
 'ان',
 'متلاشی',
 'شده_بود',
 '.',
 'که',
 'پیک',
 'زمان',
 'تحویل',
 'گفت',
 'مقداری',
 'تکان',
 'خورده',
 'وقتی',
 'دیدم',
 'کیک',
 'رو',
 'خیلی',
 'بیشتر',
 'از',
 'یه',
 'کمی',
 'بو',
 'جالبه',
 'که',
 'حتی',
 'جعبه',
 'هم',
 'متلاشی',
 'شده_بود',
 '.',
 'اگر',
 'سرویس',
 'ارسال',
 'کیک',
 'رو',
 'پذیرفتید',
 'حداقل\u200cها',
 'رو',
 'هم',
 'رعایت',
 'کنید']

In [ ]:
model = Word2Vec(sentences=sents, size=64, window=10, min_count=5, seed=42, workers=5)


In [ ]:
model['غذا']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 1.09088171e+00, -1.98121226e+00,  5.42214252e-02, -2.28773642e+00,
       -8.63751024e-02,  9.70818877e-01,  2.34429836e+00, -4.85273033e-01,
       -5.18726051e-01,  1.34508932e+00,  1.00855207e+00, -2.02378535e+00,
        1.32505965e+00, -1.24619994e-02,  6.79764330e-01, -2.10253358e+00,
       -1.57454431e+00, -1.63608015e+00,  2.14085960e+00, -2.55395222e+00,
       -2.77031612e+00,  2.21941233e+00, -2.24875259e+00, -1.13079309e+00,
        9.94559377e-02,  8.01651895e-01, -6.61697149e-01, -1.50709295e+00,
       -6.71283603e-01,  1.52014554e+00, -1.87269890e+00,  2.97042131e+00,
        2.23909998e+00, -8.68564308e-01,  3.19665819e-01, -1.36911741e-03,
       -8.80087912e-01, -2.75200391e+00,  1.47079015e+00, -2.03756973e-01,
        1.19984865e-01,  1.53575289e+00, -2.74418950e+00, -1.14466310e+00,
       -1.17781806e+00,  7.88507462e-01,  4.59679037e-01, -3.08877158e+00,
       -2.43826374e-01,  4.59297925e-01,  2.88025975e+00, -1.87719584e+00,
        1.50981450e+00,  

In [ ]:
# Let's check out most similar words to 'غذا'
model.most_similar('غذا', topn=15)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('غدا', 0.6601365804672241),
 ('غذای', 0.6189115643501282),
 ('غذاها', 0.5682851076126099),
 ('سرد', 0.5223441123962402),
 ('غذا\u200cها', 0.5147817134857178),
 ('برنج', 0.48639869689941406),
 ('جوجه', 0.475252240896225),
 ('رستوران', 0.4577062726020813),
 ('کباب', 0.4513702690601349),
 ('مکانی', 0.43562841415405273),
 ('مسافت', 0.4305807054042816),
 ('غذاش', 0.42365825176239014),
 ('کباب\u200cها', 0.4196786880493164),
 ('پیتزا', 0.4130145013332367),
 ('زمان', 0.3960300087928772)]

In [ ]:
# Another example
model.most_similar('عالی', topn=15)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('لذیذ', 0.794096827507019),
 ('عالى', 0.7726255655288696),
 ('خوشمزه', 0.7691041231155396),
 ('خوش', 0.7538832426071167),
 ('عاااالی', 0.7529487609863281),
 ('خوب', 0.7189466953277588),
 ('عالیه', 0.7039587497711182),
 ('عااالی', 0.6689847707748413),
 ('باکیفیت', 0.6586582660675049),
 ('مرسی', 0.6434277296066284),
 ('تمیز', 0.6419634819030762),
 ('چسبید', 0.6129447221755981),
 ('بینظیر', 0.6064640879631042),
 ('ممنونم', 0.6057639718055725),
 ('خوشمزه\u200cای', 0.5977407693862915)]

In [ ]:
print(model.wv.vocab)

{'واقعا': <gensim.models.keyedvectors.Vocab object at 0x7f6d90e9af10>, 'حیف': <gensim.models.keyedvectors.Vocab object at 0x7f6d90eaee10>, 'وقت': <gensim.models.keyedvectors.Vocab object at 0x7f6d90eaecd0>, 'که': <gensim.models.keyedvectors.Vocab object at 0x7f6d90e9edd0>, 'بنویسم': <gensim.models.keyedvectors.Vocab object at 0x7f6d90e9e410>, 'سرویس': <gensim.models.keyedvectors.Vocab object at 0x7f6d90e9e910>, 'دهیتون': <gensim.models.keyedvectors.Vocab object at 0x7f6d90e9e6d0>, 'شده': <gensim.models.keyedvectors.Vocab object at 0x7f6d90e9e8d0>, 'افتضاح': <gensim.models.keyedvectors.Vocab object at 0x7f6d90e9e7d0>, 'قرار': <gensim.models.keyedvectors.Vocab object at 0x7f6d90e9e2d0>, 'بود': <gensim.models.keyedvectors.Vocab object at 0x7f6d90e9ed50>, '۱': <gensim.models.keyedvectors.Vocab object at 0x7f6d90e9e290>, 'ساعته': <gensim.models.keyedvectors.Vocab object at 0x7f6d90e9e610>, 'برسه': <gensim.models.keyedvectors.Vocab object at 0x7f6d90e9e950>, 'ولی': <gensim.models.keyedvector

In [ ]:
# total numberof extracted words.
vocab=model.wv.vocab
print("The total number of words are : ",len(vocab))

The total number of words are :  7416


In [ ]:
# print(vocab)
vocab=list(vocab.keys())

In [ ]:
word_vec_dict={}
for word in vocab:
  word_vec_dict[word]=model.wv.get_vector(word)
print("The no of key-value pairs : ",len(word_vec_dict)) # should come equal to vocab size

The no of key-value pairs :  7416


In [ ]:
tok = Tokenizer()
tok.fit_on_texts(train['comment'])
vocab_size = len(tok.word_index) + 1
encd_rev = tok.texts_to_sequences(train['comment'])

In [ ]:
max_rev_len=1565  # max lenght of a comment
vocab_size = len(tok.word_index) + 1  # total no of words
embed_dim=64 # embedding dimension as choosen in word2vec constructor

In [ ]:
# now padding to have a amximum length of 1565
pad_rev= pad_sequences(encd_rev, maxlen=max_rev_len, padding='post')
pad_rev.shape   # note that we had 100K reviews and we have padded each review to have  a lenght of 1565 words.

(56700, 1565)

In [ ]:
# now creating the embedding matrix
embed_matrix=np.zeros(shape=(vocab_size,embed_dim))
for word,i in tok.word_index.items():
  embed_vector=word_vec_dict.get(word)
  if embed_vector is not None:  # word is in the vocabulary learned by the w2v model
    embed_matrix[i]=embed_vector
  # if word is not found then embed_vector corressponding to that vector will stay zero.

In [ ]:
# checking.
print(embed_matrix[14])

[-0.95929635 -2.07674766 -0.35118034  0.67105216  0.67793369  0.68923932
  1.72554588 -1.5454663   1.82808578  1.55060804  0.16747724 -1.00659025
 -2.65731454  1.70247936  0.24443905  0.09838589 -2.10065413  1.57566762
 -1.6256516  -3.94431686 -0.88075268  0.34473836 -0.97776425 -1.33720458
  1.03059459 -3.33678508  0.32726604  1.16154706 -3.05139136  0.89104986
  3.2579577   0.17585397 -2.97154546 -0.77241677  0.63807583 -1.1169498
 -0.94570589 -0.85519451  0.35898116 -1.66302705  0.1596479   1.07854772
  4.02586079 -1.06827974  0.58745253  0.84967065 -1.52787352 -1.19965065
 -2.05820632 -2.80139971  1.48746765  0.87319094 -1.10732746 -0.09159309
 -0.60718054 -0.21751478  1.59093678 -1.55177522  2.3060863  -0.68398166
  0.60235029  0.87529701  2.9110384   0.20176001]


In [ ]:
# features = df.columns.tolist()[:-1]
features = ['comment']
label = ['label']

X_train, X_test, Y_train, Y_test = train_test_split(train[features], train[label], test_size=0.3)

X_train = [text for text in X_train.comment]
Y_train = [label for label in Y_train.label]

X_test = [text for text in X_test.comment]
Y_test = [label for label in Y_test.label]

In [ ]:
# prepare train and val sets first
Y= to_categorical(train['label'])    # one hot target as required by NN.
x_train,x_test,y_train,y_test=train_test_split(pad_rev,Y,test_size=0.20,random_state=42)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=embed_dim,input_length=max_rev_len,embeddings_initializer=Constant(embed_matrix)))
# model.add(CuDNNLSTM(64,return_sequences=False)) # loss stucks at about 
model.add(Flatten())
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.50))
# model.add(Dense(16,activation='relu'))
# model.add(Dropout(0.20))
model.add(Dense(2,activation='sigmoid'))  # sigmod for bin. classification.

In [ ]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 1565, 64)          2186560   
                                                                 
 flatten_6 (Flatten)         (None, 100160)            0         
                                                                 
 dense_13 (Dense)            (None, 16)                1602576   
                                                                 
 dropout_5 (Dropout)         (None, 16)                0         
                                                                 
 dense_14 (Dense)            (None, 2)                 34        
                                                                 
Total params: 3,789,170
Trainable params: 3,789,170
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# compile the model
model.compile(optimizer=optimizers.RMSprop(lr=1e-3),loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
# specify batch size and epocj=hs for training.
epochs=7
batch_size=64

In [ ]:
  # fitting the model.
model.fit(x_train,y_train,epochs=epochs,batch_size=batch_size,validation_data=(x_test,y_test))

Epoch 1/7
709/709 [==============================] - 46s 64ms/step - loss: 0.4468 - accuracy: 0.8049 - val_loss: 0.3804 - val_accuracy: 0.8377
Epoch 2/7
709/709 [==============================] - 43s 61ms/step - loss: 0.4001 - accuracy: 0.8281 - val_loss: 0.3743 - val_accuracy: 0.8416
Epoch 3/7
709/709 [==============================] - 43s 61ms/step - loss: 0.3713 - accuracy: 0.8431 - val_loss: 0.3686 - val_accuracy: 0.8425
Epoch 4/7
709/709 [==============================] - 48s 67ms/step - loss: 0.3565 - accuracy: 0.8517 - val_loss: 0.3724 - val_accuracy: 0.8417
Epoch 5/7
709/709 [==============================] - 46s 65ms/step - loss: 0.3456 - accuracy: 0.8570 - val_loss: 0.3701 - val_accuracy: 0.8457
Epoch 6/7
709/709 [==============================] - 49s 69ms/step - loss: 0.3342 - accuracy: 0.8601 - val_loss: 0.3824 - val_accuracy: 0.8422
Epoch 7/7
709/709 [==============================] - 45s 64ms/step - loss: 0.3245 - accuracy: 0.8645 - val_loss: 0.3828 - val_accuracy: 0.8391

In [ ]:
output_dir='model_output/lstm/'

epochs = 8
batch_size = 128

n_dim = 64
n_unique_words = 5000
max_text_length = 500
pad_type = trunc_type = 'pre'
drop_embed = 0.2

n_lstm = 256
drop_lstm = 0.2

In [ ]:
tok = Tokenizer(num_words=n_unique_words, lower=False)
tok.fit_on_texts(X_train)

train_sequences = tok.texts_to_sequences(X_train)
train_sequences_matrix = pad_sequences(train_sequences, maxlen=max_text_length)

test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences, maxlen=max_text_length)

In [ ]:
train_sequences_matrix.shape


(39690, 500)

In [ ]:
def LSTM_NN():
    model = Sequential()
    model.add(Embedding(n_unique_words, n_dim, input_length=max_text_length))
    model.add(SpatialDropout1D(drop_embed))
    model.add(LSTM(n_lstm, dropout=drop_lstm))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [ ]:
model = LSTM_NN()
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 500, 64)           320000    
                                                                 
 spatial_dropout1d_4 (Spatia  (None, 500, 64)          0         
 lDropout1D)                                                     
                                                                 
 lstm_4 (LSTM)               (None, 256)               328704    
                                                                 
 dense_4 (Dense)             (None, 1)                 257       
                                                                 
Total params: 648,961
Trainable params: 648,961
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model_ckpt = ModelCheckpoint(filepath=output_dir+'weights.{epoch:02d}.hdf5')

test_sequences_matrix = np.array(test_sequences_matrix)
Y_test = np.array(Y_test)

In [ ]:
type(Y_test)

numpy.ndarray

In [ ]:
model.fit(train_sequences_matrix, Y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          verbose=1,
          validation_data=(test_sequences_matrix, Y_test), callbacks=[model_ckpt])

ValueError: ignored

In [ ]:
max_words = 5000
max_len = 200
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
comments = pad_sequences(sequences, maxlen=max_len)
print(comments)

[[   0    0    0 ... 4320   14  160]
 [   0    0    0 ...   49  556 1203]
 [   0    0    0 ...  434    2  161]
 ...
 [   0    0    0 ...    7 2652 1469]
 [   0    0    0 ...   44 2548  856]
 [   0    0    0 ...   17  410  169]]


In [ ]:
w2v_model=gensim.models.Word2Vec(sentences=data,size=300,window=10)

In [ ]:
# total numberof extracted words.
vocab=w2v_model.wv.vocab
print("The total number of words are : ",len(vocab))
print(vocab)

The total number of words are :  137
{'و': <gensim.models.keyedvectors.Vocab object at 0x7f6d9752fb10>, 'ا': <gensim.models.keyedvectors.Vocab object at 0x7f6d95b4e650>, 'ق': <gensim.models.keyedvectors.Vocab object at 0x7f6d95b4ef90>, 'ع': <gensim.models.keyedvectors.Vocab object at 0x7f6d98709990>, ' ': <gensim.models.keyedvectors.Vocab object at 0x7f6d98709890>, 'ح': <gensim.models.keyedvectors.Vocab object at 0x7f6d95b79710>, 'ی': <gensim.models.keyedvectors.Vocab object at 0x7f6d95b87810>, 'ف': <gensim.models.keyedvectors.Vocab object at 0x7f6d95b87ed0>, 'ت': <gensim.models.keyedvectors.Vocab object at 0x7f6d98709e90>, 'ک': <gensim.models.keyedvectors.Vocab object at 0x7f6d95becb10>, 'ه': <gensim.models.keyedvectors.Vocab object at 0x7f6d95b87710>, 'ب': <gensim.models.keyedvectors.Vocab object at 0x7f6d95b87450>, 'ن': <gensim.models.keyedvectors.Vocab object at 0x7f6d95b87fd0>, 'س': <gensim.models.keyedvectors.Vocab object at 0x7f6d95b87390>, 'م': <gensim.models.keyedvectors.Vocab

In [ ]:
embedding_layer = Embedding(1000, 64)


In [ ]:
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

NameError: ignored

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [ ]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])

texts:  [b'"Magnolia" is a preposterous, bewildering acting showcase that adds up to very little. Like "Eyes Wide Shut," "Magnolia" is an aimless series of episodes without any concern for coherence. The camera swoops through hallways and corridors, catching glimpses of sad characters. Where is the reason to care for these people? The common theme seems to be people who yell a lot, who can\'t care for others (except for John C. Reilly\'s and Philip Seymour Hoffman\'s characters), and are self-destructive jerks who are either falling to pieces or dying. I was reminded of how much I disliked "Shine" because of the irredeemable monster of a father played by Armin Mueller-Stahl. There are so many unattractive, unappealing characters here, why would we want to spend time with them?<br /><br />Having said that, there is nothing held back about "Magnolia." Paul Thomas Anderson\'s ideas are splashed onto his canvas with abandon. There are two ideas in particular that bomb. Both happen in the l

In [ ]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [ ]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U14')

In [ ]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[  1,   7,   4, ...,   0,   0,   0],
       [ 10,  67, 322, ...,   0,   0,   0],
       [ 45, 320,   2, ...,   0,   0,   0]])

In [ ]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

Original:  b'"Magnolia" is a preposterous, bewildering acting showcase that adds up to very little. Like "Eyes Wide Shut," "Magnolia" is an aimless series of episodes without any concern for coherence. The camera swoops through hallways and corridors, catching glimpses of sad characters. Where is the reason to care for these people? The common theme seems to be people who yell a lot, who can\'t care for others (except for John C. Reilly\'s and Philip Seymour Hoffman\'s characters), and are self-destructive jerks who are either falling to pieces or dying. I was reminded of how much I disliked "Shine" because of the irredeemable monster of a father played by Armin Mueller-Stahl. There are so many unattractive, unappealing characters here, why would we want to spend time with them?<br /><br />Having said that, there is nothing held back about "Magnolia." Paul Thomas Anderson\'s ideas are splashed onto his canvas with abandon. There are two ideas in particular that bomb. Both happen in the

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [ ]:
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True]


In [ ]:
# predict on a sample text without padding.

sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

[-0.01035371]


In [ ]:
# predict on a sample text with padding

padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

[-0.01035372]


In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

Epoch 1/10
391/391 [==============================] - 671s 2s/step - loss: 0.6321 - accuracy: 0.5791 - val_loss: 0.4540 - val_accuracy: 0.7844
Epoch 2/10
391/391 [==============================] - 656s 2s/step - loss: 0.3907 - accuracy: 0.8217 - val_loss: 0.3652 - val_accuracy: 0.8276
Epoch 3/10
391/391 [==============================] - 647s 2s/step - loss: 0.3339 - accuracy: 0.8523 - val_loss: 0.3354 - val_accuracy: 0.8620
Epoch 4/10
391/391 [==============================] - 657s 2s/step - loss: 0.3188 - accuracy: 0.8618 - val_loss: 0.3462 - val_accuracy: 0.8583
Epoch 5/10
391/391 [==============================] - 659s 2s/step - loss: 0.3103 - accuracy: 0.8650 - val_loss: 0.3249 - val_accuracy: 0.8578
Epoch 6/10
391/391 [==============================] - 649s 2s/step - loss: 0.3087 - accuracy: 0.8660 - val_loss: 0.3334 - val_accuracy: 0.8443
Epoch 7/10
391/391 [==============================] - 655s 2s/step - loss: 0.3045 - accuracy: 0.8678 - val_loss: 0.3228 - val_accuracy: 0.8495